In [1]:
import subprocess
import sys
from pathlib import Path
import errors
from Lexer.Lexer_generator import Lexer
from Parser.Parser import ParserError, LR1Parser
from Parser.HulkParser import HulkParser
from Grammar.Grammar import G
from Grammar import Grammar as Gr
from Lexer.Tokens_lexer import tokens
from Semantic.semantic_analysis_pipeline import semantic_analysis_pipeline
from Code_gen.code_generator import CCodeGenerator
from cmp.evaluation import evaluate_reverse_parse
import os




In [2]:
parser=LR1Parser(G) 

In [3]:
lexer= Lexer(tokens,G.EOF)

In [7]:
def print_error(message):
    red = "\033[31m"
    refresh = "\033[0m"
    print(f"{red}{message}{refresh}")

text= '''
type A inherits B {
}
type B inherits A {

}
5;
            '''

try:
    tokens_=lexer.tokenize(text)
except errors.LexerError as e:
    print_error(e)

try: 
    parse, operations = parser([t.token_type for t in tokens_])
except ParserError as e:
    error_token = tokens_[e.token_index]
    error_text = errors.HulkSyntacticError.PARSING_ERROR % error_token.lex
    error_ = [errors.HulkSyntacticError(error_text, error_token.row, error_token.column,)]
    print_error(error_)
    
ast=evaluate_reverse_parse(parse,operations,tokens_)
semantic_ast, semantic_errors,context,_=semantic_analysis_pipeline(ast)
if semantic_errors:
    for e in semantic_errors:
        print_error(e)
    
codegen=CCodeGenerator()
code= codegen(semantic_ast,context)
try:
    with open(Path(f'{"archivo"}.c'), 'w') as f:
        f.write(code)
except FileNotFoundError:
    error = errors.HulkIOError(errors.HulkIOError.ERROR_WRITING_FILE % output_path)
    print_error(error)
    
subprocess.run(["gcc","-o",'archivo','archivo.c', "-lm"])
subprocess.run(['./archivo'])

SemanticError: Circular dependency inheritance. Near line: 3, column: 17


CompletedProcess(args=['./archivo'], returncode=0)

In [11]:
class A:
    x="5"

class B(A):
    y="4"

p=B()
p.x="9"
p.x

'9'